In [2]:
import urllib
from flask import abort, request, Flask
import requests
import json

In [3]:
client_id = 'aff8e8e9-0f5d-4fb7-8f9f-ce19fbef6b5c'
client_secret = json.load(open('secret.json'))['client_secret']
redirect_uri = "http://127.0.0.1:5000/callback"
base_url = "https://simulator-api.db.com/gw/oidc/authorize"

In [7]:
app = Flask(__name__)

# Authorization
@app.route("/")
def homepage():
    text = '<a href="%s">Authenticate with Deutsche Bank</a>'
    return text % make_authorization_url()

def make_authorization_url():
    params = {"client_id": client_id,
              "response_type": "code",
              "state": redirect_uri,
              "redirect_uri": redirect_uri}
    import urllib
    url = base_url + "?" + urllib.urlencode(params)
    return url

@app.route('/callback')
def deutsche_callback():
    error = request.args.get('error', '')
    if error:
        return "Error: " + error
    code = request.args.get('code')
    access_token = get_token(code)
    return get_transactions(access_token)


def get_token(code):
    client_auth = requests.auth.HTTPBasicAuth(client_id, client_secret)
    post_data = {"grant_type": "authorization_code",
                 "code": code,
                 "redirect_uri": redirect_uri}
    response = requests.post("https://simulator-api.db.com/gw/oidc/token",
                             auth=client_auth,
                             data=post_data)
    token_json = response.json()
    return token_json["access_token"]


def get_cashAccounts(access_token):
    trans_url = 'https://simulator-api.db.com/gw/dbapi/v1/cashAccounts'
    headers = {"Authorization": "bearer " + access_token}
    response = requests.get(trans_url, headers=headers)
    response = response.json()
    s = ''
    for i in range(len(response)):
        s += "iban: "+response[i]['iban'] + "<br/> balance: " + str(response[i]['balance']) +\
            '<br/> productDescription: ' + response[i]['productDescription'] + '<br/><br/>' 
    return response

def get_transactions(access_token):
    trans_url = 'https://simulator-api.db.com/gw/dbapi/v1/transactions'
    headers = {"Authorization": "bearer " + access_token}
    response = requests.get(trans_url, headers=headers)
    response = response.json()
    s = ''
    for i in range(len(response)):
        s += "iban: "+response[i]['originIban'] + "<br/> amount: " + str(response[i]['amount']) +\
            '<br/> bookingDate: ' + response[i]['bookingDate'] 
        if 'counterPartyName' in response[i]:
            s += '<br/> counterPartyName: ' + response[i]['counterPartyName']
        if 'counterPartyIban' in response[i]:
            s += '<br/> counterPartyIban: ' + response[i]['counterPartyIban']
        if 'usage' in response[i]:
            s += '<br/> usage: ' + response[i]['usage']            
        s += '<br/><br/>' 
        
    s += '<br/><br/>' 
    cashAccounts = get_cashAccounts(access_token)
    for i in range(len(cashAccounts)):
        s += "iban: "+cashAccounts[i]['iban'] + "<br/> balance: " + str(cashAccounts[i]['balance']) +\
            '<br/> productDescription: ' + cashAccounts[i]['productDescription'] + '<br/><br/>'
    return s

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Mar/2017 05:13:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2017 05:13:57] "GET /callback?code=4SebZR&state=http://127.0.0.1:5000/callback HTTP/1.1" 200 -
